# Loading ENV

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

hello world


In [ ]:
from path import Path

MODEL_DIR: Path = Path(os.environ['MODEL_DIR'])
FINE_TUNE_DIR: str = MODEL_DIR/"fine_tuned"

# Load DeBERTaV3 Model